In [1]:
import pandas as pd

In [2]:
biz_df = pd.read_csv('biz_cluster.csv')

In [6]:
biz_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57173 entries, 0 to 57172
Data columns (total 24 columns):
Unnamed: 0                57173 non-null int64
business_id               57173 non-null object
RestaurantsPriceRange2    51111 non-null float64
stars                     57173 non-null float64
review_count              57173 non-null int64
categories                57173 non-null object
cl_0                      25785 non-null float64
cl_1                      37611 non-null float64
cl_2                      6306 non-null float64
cl_3                      327 non-null float64
cl_4                      11547 non-null float64
cl_5                      14129 non-null float64
cl_6                      1 non-null float64
cl_7                      3366 non-null float64
cl_8                      2667 non-null float64
cl_9                      2073 non-null float64
cl_10                     380 non-null float64
cl_11                     2937 non-null float64
cl_12                     47

In [7]:
biz_df.head().T

,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
business_id,--9e1ONYQuAa-CB_Rrw7Tw,-VAsjhmAbKF3Pb_-8rh3xg,-cxD1NimFldATDUsN-oa3A,-r8SvItXXG6_T3mP5GXRAw,0859wfd1BQHG46Zpwhc0ZQ
RestaurantsPriceRange2,4,1,2,2,2
stars,4,2,2,4,4.5
review_count,1546,10,23,10,245
categories,"Steakhouses, Restaurants, Cajun/Creole","Fast Food, Burgers, Restaurants","Mexican, Restaurants","Restaurants, Noodles, Food, Cafes, Chinese, Co...","Nightlife, Pizza, Wine Bars, Bars, American (N..."
cl_0,0,NaN,NaN,NaN,NaN
cl_1,1,1,1,NaN,1
cl_2,2,NaN,NaN,NaN,NaN
cl_3,NaN,NaN,NaN,NaN,NaN


In [9]:
len(biz_df.categories.unique())

27807